In [1]:
import os 
import pickle 
from tqdm.notebook import tqdm 
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout,add
import gdown

In [2]:
%pwd

'/Users/rohansridhar/Desktop/caption_gen/IMAGE-CAPTION_GENERATOR/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rohansridhar/Desktop/caption_gen/IMAGE-CAPTION_GENERATOR'

In [5]:
model=VGG16()
model=Model(inputs=model.inputs,outputs=model.layers[-2].output)
print(model.summary())

2024-02-15 18:12:46.466495: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-15 18:12:46.466540: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-15 18:12:46.466551: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-15 18:12:46.466663: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-15 18:12:46.467013: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
BASE_DIR='artifacts/data_ingestion'

In [7]:
#extract features from image 
features={}
directory=os.path.join(BASE_DIR,'Images')
for img_name in tqdm(os.listdir(directory)):
    img_path= directory + '/' + img_name
    image=load_img(img_path,target_size=(224,224))
    #pixels to numpy array
    image=img_to_array(image)
    #reshape data
    image=image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    image=preprocess_input(image)
    #extract features
    feature=model.predict(image,verbose=0)
    #get iamge id 
    image_id=img_name.split('.')[0]
    #store features
    features[image_id]=feature
    
    

  0%|          | 0/8091 [00:00<?, ?it/s]

2024-02-15 18:12:51.067596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [8]:
#store features in pixel 
WORKING_DIR='artifacts/data_ingestion'
pickle.dump(features, open(os.path.join(WORKING_DIR,"features.pkl"), "wb"))

In [9]:
#load pickle 
with open(os.path.join(WORKING_DIR,"features.pkl"), "rb") as f:
    features=pickle.load(f)

In [10]:
#load the captions
with open(os.path.join(BASE_DIR,'captions.txt'),'r') as f:
    next(f)
    captions_doc=f.read()
    

In [11]:
#map image to captions 
mapping={}
for line in tqdm(captions_doc.split('\n')):
    tokens=line.split(',')
    if len(line) < 2:
        continue 
    image_id,caption=tokens[0],tokens[1:]
    #remove extensions 
    image_id = os.path.split('.')[0]
    #caption list to string 
    caption=" ".join(caption)
    #create list 
    if image_id not in mapping:
        mapping[image_id]=[]
    mapping[image_id].append(caption)
    

  0%|          | 0/40456 [00:00<?, ?it/s]

In [12]:
len(mapping)

1

In [24]:
def clean(mapping):
    for key,captions in mapping.items():
        for i in range(len(captions)):
            caption=captions[i]
            caption=caption.lower()
            caption=caption.replace('[^A-Za-z]','')
            caption=caption.replace('\s+', ' ')
            #adding tags 
            caption='<start> ' + " ".join([word for word in caption.split() if len(word)>1]) + ' <end>'
            caption[i]=caption

In [26]:
mapping['1000268201_693b08cb0e']

KeyError: '1000268201_693b08cb0e'